In [ ]:
# 保存为项目所需格式：300809.sz.csv
# 注意：文件名使用小写 sz

# 提取股票代码的数字和后缀部分
code_number = TS_CODE.split('.')[0]  # '300809'
code_suffix = TS_CODE.split('.')[1].lower()  # 'sz'

# 构建文件名
output_filename = f'data/{code_number}.{code_suffix}.csv'

# 保存合并后的数据
df_merged.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("="*60)
print(f"✅ 项目格式数据已保存: {output_filename}")
print(f"   文件大小: {os.path.getsize(output_filename) / 1024:.2f} KB")
print(f"   记录数: {len(df_merged)}")
print(f"   日期范围: {df_merged['trade_date'].min()} ~ {df_merged['trade_date'].max()}")
print("="*60)
print("\n🎉 现在可以在项目中使用该股票数据了！")
print(f"   股票代码: {TS_CODE}")
print(f"   数据文件: {output_filename}")

## 11. 保存为项目所需格式（300809.sz.csv）

# 000155 股票数据预处理

本 Notebook 用于从 Tushare 拉取 000155（中联重科）的**全部历史日线数据**和 **daily_basic 数据**，并保存到本地文件。

## 1. 导入依赖包

```bash
pip install tushare pandas
```

In [ ]:
import tushare as ts
import pandas as pd
from datetime import datetime, timedelta
import time
import os

print(f"tushare version: {ts.__version__}")
print(f"pandas version: {pd.__version__}")

## 2. 配置 Tushare Token

**请替换为你的 Tushare Pro Token**

获取方式：https://tushare.pro/register

In [ ]:
# ⚠️ 请替换为你的 Token
TOKEN = 'your_tushare_token_here'

# 初始化 Tushare Pro API
pro = ts.pro_api(TOKEN)

# 股票代码
TS_CODE = '000155.SZ'  # 中联重科

# 使用通用的早期日期，无需手动查找上市日期
# 1990-01-01 早于所有A股上市日期，API会自动从实际上市日开始返回数据
START_DATE = '19900101'

print(f"股票代码: {TS_CODE}")
print(f"查询起始日期: {START_DATE}（实际从上市日开始）")

## 3. 定义数据获取函数

In [ ]:
def get_all_daily_data(ts_code, start_date, sleep_time=0.3):
    """
    分批获取全部历史日线数据
    
    Parameters:
    -----------
    ts_code : str
        股票代码，如 '000155.SZ'
    start_date : str
        开始日期，格式 'YYYYMMDD'
    sleep_time : float
        每次请求间隔时间（秒），避免请求过快
    
    Returns:
    --------
    pd.DataFrame
        包含全部历史日线数据的 DataFrame
    """
    all_data = []
    
    # 计算日期范围
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.now()
    
    current_start = start
    
    print(f"开始获取日线数据: {start_date} ~ {end.strftime('%Y%m%d')}")
    print("=" * 60)
    
    batch_count = 0
    
    # 每次获取1年的数据
    while current_start < end:
        current_end = min(current_start + timedelta(days=365), end)
        
        start_str = current_start.strftime('%Y%m%d')
        end_str = current_end.strftime('%Y%m%d')
        
        try:
            df = pro.daily(
                ts_code=ts_code,
                start_date=start_str,
                end_date=end_str
            )
            
            if not df.empty:
                all_data.append(df)
                batch_count += 1
                print(f"[{batch_count:2d}] {start_str} ~ {end_str}: {len(df):4d} 条")
            else:
                print(f"[{batch_count:2d}] {start_str} ~ {end_str}: 无数据")
            
            time.sleep(sleep_time)  # 避免请求过快
            
        except Exception as e:
            print(f"❌ 获取数据失败 [{start_str} ~ {end_str}]: {e}")
        
        current_start = current_end + timedelta(days=1)
    
    print("=" * 60)
    
    # 合并所有数据
    if all_data:
        result = pd.concat(all_data, ignore_index=True)
        # 去重并排序
        result = result.drop_duplicates(subset=['trade_date']).sort_values('trade_date')
        print(f"✅ 日线数据获取完成，共 {len(result)} 条记录")
        return result
    else:
        print("❌ 未获取到任何数据")
        return pd.DataFrame()

In [ ]:
def get_all_daily_basic(ts_code, start_date, sleep_time=0.3):
    """
    分批获取全部历史 daily_basic 数据（包含流通股本等指标）
    
    Parameters:
    -----------
    ts_code : str
        股票代码，如 '000155.SZ'
    start_date : str
        开始日期，格式 'YYYYMMDD'
    sleep_time : float
        每次请求间隔时间（秒）
    
    Returns:
    --------
    pd.DataFrame
        包含全部历史 daily_basic 数据的 DataFrame
    """
    all_data = []
    
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.now()
    
    current_start = start
    
    print(f"开始获取 daily_basic 数据: {start_date} ~ {end.strftime('%Y%m%d')}")
    print("=" * 60)
    
    batch_count = 0
    
    while current_start < end:
        current_end = min(current_start + timedelta(days=365), end)
        
        start_str = current_start.strftime('%Y%m%d')
        end_str = current_end.strftime('%Y%m%d')
        
        try:
            df = pro.daily_basic(
                ts_code=ts_code,
                start_date=start_str,
                end_date=end_str,
                # 注意：移除了 close 字段，因为 daily 接口已经提供了
                fields='ts_code,trade_date,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv'
            )
            
            if not df.empty:
                all_data.append(df)
                batch_count += 1
                print(f"[{batch_count:2d}] {start_str} ~ {end_str}: {len(df):4d} 条")
            else:
                print(f"[{batch_count:2d}] {start_str} ~ {end_str}: 无数据")
            
            time.sleep(sleep_time)
            
        except Exception as e:
            print(f"❌ 获取数据失败 [{start_str} ~ {end_str}]: {e}")
        
        current_start = current_end + timedelta(days=1)
    
    print("=" * 60)
    
    if all_data:
        result = pd.concat(all_data, ignore_index=True)
        result = result.drop_duplicates(subset=['trade_date']).sort_values('trade_date')
        print(f"✅ daily_basic 数据获取完成，共 {len(result)} 条记录")
        return result
    else:
        print("❌ 未获取到任何数据")
        return pd.DataFrame()

## 4. 获取日线数据

In [ ]:
# 获取全部历史日线数据
df_daily = get_all_daily_data(TS_CODE, START_DATE, sleep_time=0.3)

# 查看数据
print("\n数据预览:")
display(df_daily.head())
print("\n数据信息:")
print(df_daily.info())

## 5. 获取 daily_basic 数据

In [ ]:
# 获取全部历史 daily_basic 数据
df_basic = get_all_daily_basic(TS_CODE, START_DATE, sleep_time=0.3)

# 查看数据
print("\n数据预览:")
display(df_basic.head())
print("\n数据信息:")
print(df_basic.info())

## 6. 合并数据

In [ ]:
# 合并日线数据和 basic 数据
df_merged = pd.merge(
    df_daily, 
    df_basic, 
    on=['ts_code', 'trade_date'], 
    how='left',
    suffixes=('', '_basic')
)

print(f"✅ 数据合并完成")
print(f"   - 日线数据: {len(df_daily)} 条")
print(f"   - basic数据: {len(df_basic)} 条")
print(f"   - 合并后: {len(df_merged)} 条")

# 查看合并后的数据
print("\n合并后数据预览:")
display(df_merged.head())

# 查看列名
print("\n列名:")
print(df_merged.columns.tolist())

## 7. 数据质量检查

In [ ]:
print("="*60)
print("数据质量检查报告")
print("="*60)

# 基本统计
print(f"\n1. 基本信息")
print(f"   股票代码: {TS_CODE}")
print(f"   数据起始: {df_merged['trade_date'].min()}")
print(f"   数据终止: {df_merged['trade_date'].max()}")
print(f"   总记录数: {len(df_merged)}")

# 缺失值检查
print(f"\n2. 缺失值统计")
missing = df_merged.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    print(missing)
else:
    print("   ✅ 无缺失值")

# 重复值检查
print(f"\n3. 重复值检查")
duplicates = df_merged.duplicated(subset=['trade_date']).sum()
print(f"   重复日期数: {duplicates}")

# 数据范围
print(f"\n4. 价格数据统计")
print(f"   最高价: {df_merged['high'].max():.2f}")
print(f"   最低价: {df_merged['low'].min():.2f}")
print(f"   平均收盘价: {df_merged['close'].mean():.2f}")

# 成交量统计
print(f"\n5. 成交量统计")
print(f"   平均成交量: {df_merged['vol'].mean()/10000:.2f} 万手")
print(f"   最大成交量: {df_merged['vol'].max()/10000:.2f} 万手")

# 股本统计（如果有）
if 'total_share' in df_merged.columns:
    print(f"\n6. 股本统计（最新）")
    latest = df_merged.iloc[-1]
    print(f"   总股本: {latest['total_share']:.2f} 万股")
    if pd.notna(latest.get('float_share')):
        print(f"   流通股本: {latest['float_share']:.2f} 万股")
    if pd.notna(latest.get('free_share')):
        print(f"   自由流通股本: {latest['free_share']:.2f} 万股")

print("\n" + "="*60)

## 8. 保存数据到本地

将数据保存为多种格式，方便后续使用

In [ ]:
# 创建 data 目录（如果不存在）
os.makedirs('data', exist_ok=True)

# 文件路径
csv_path = f'data/{TS_CODE.replace(".", "_")}_full_history.csv'
csv_daily_path = f'data/{TS_CODE.replace(".", "_")}_daily.csv'
csv_basic_path = f'data/{TS_CODE.replace(".", "_")}_basic.csv'

# 保存合并数据
print("开始保存数据...")
print("="*60)

# 1. 保存合并后的完整数据
df_merged.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"✅ 合并数据已保存: {csv_path}")
print(f"   文件大小: {os.path.getsize(csv_path) / 1024:.2f} KB")

# 2. 分别保存日线和 basic 数据
df_daily.to_csv(csv_daily_path, index=False, encoding='utf-8-sig')
print(f"\n✅ 日线数据已保存: {csv_daily_path}")
print(f"   文件大小: {os.path.getsize(csv_daily_path) / 1024:.2f} KB")

df_basic.to_csv(csv_basic_path, index=False, encoding='utf-8-sig')
print(f"\n✅ Basic数据已保存: {csv_basic_path}")
print(f"   文件大小: {os.path.getsize(csv_basic_path) / 1024:.2f} KB")

print("\n" + "="*60)
print("🎉 所有数据保存完成！")

## 9. 测试读取保存的数据

In [ ]:
# 测试读取保存的数据
test_df = pd.read_csv(csv_path)

print(f"✅ 数据读取成功！")
print(f"   记录数: {len(test_df)}")
print(f"   列数: {len(test_df.columns)}")
print("\n前5条记录:")
display(test_df.head())
print("\n后5条记录:")
display(test_df.tail())

## 10. 数据说明

### 保存的文件

1. **`data/000155_SZ_full_history.csv`** - 完整数据（日线 + basic 合并）
2. **`data/000155_SZ_daily.csv`** - 仅日线数据
3. **`data/000155_SZ_basic.csv`** - 仅 basic 数据

### 主要字段说明

#### 日线数据字段：
- `ts_code`: 股票代码
- `trade_date`: 交易日期
- `open`: 开盘价
- `high`: 最高价
- `low`: 最低价
- `close`: 收盘价
- `pre_close`: 昨收价
- `change`: 涨跌额
- `pct_chg`: 涨跌幅 (%)
- `vol`: 成交量（手）
- `amount`: 成交额（千元）

#### Basic 数据字段：
- `turnover_rate`: 换手率 (%)
- `turnover_rate_f`: 换手率（自由流通股） (%)
- `volume_ratio`: 量比
- `pe`: 市盈率（总市值/净利润）
- `pe_ttm`: 市盈率TTM
- `pb`: 市净率（总市值/净资产）
- `ps`: 市销率
- `ps_ttm`: 市销率TTM
- `dv_ratio`: 股息率 (%)
- `dv_ttm`: 股息率TTM (%)
- `total_share`: 总股本（万股）
- `float_share`: 流通股本（万股）
- `free_share`: 自由流通股本（万股）
- `total_mv`: 总市值（万元）
- `circ_mv`: 流通市值（万元）

### 后续使用

```python
# 读取完整数据
df = pd.read_csv('data/000155_SZ_full_history.csv')

# 转换日期格式
df['trade_date'] = pd.to_datetime(df['trade_date'])

# 按日期排序
df = df.sort_values('trade_date')
```